<a href="https://colab.research.google.com/github/sarthak221995/100DaysOfMLCode/blob/master/LinearRegressionImplementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LINEAR REGRESSION FROM SCRATCH

In [1]:
import random 
import torch
import matplotlib.pyplot as plt

%matplotlib inline

In [8]:
# CREATE SYNTHETIC DATA

def synthetic_data(w, b, num_examples):
  """Generate y = Xw + b + noise"""
  X = torch.normal(0, 1, (num_examples, len(w)))
  y = torch.matmul(X, w) + b
  y+= torch.normal(0,0.01, y.shape)
  return X, y.reshape((-1, 1))

true_w = torch.tensor([2,-3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)
print(f"Features Shape :{features.shape}\n Labels Shape :{labels.shape}")

Features Shape :torch.Size([1000, 2])
 Labels Shape :torch.Size([1000, 1])


In [35]:
# CREATE MINIBATCH

def data_iter(batch_size, features, labels):
  num_examples = len(features)
  indices = list(range(num_examples))
  random.shuffle(indices)
  for i in range(0, num_examples, batch_size):
    batch_indices = torch.tensor(indices[i:min(i+batch_size,num_examples)])
    yield features[batch_indices],labels[batch_indices]
  

In [30]:
# INITIALIZE MODEL PARAMETERS

w = torch.normal(0,0.01, size=(2,1),requires_grad=True)
b = torch.zeros(1,requires_grad=True)

In [31]:
# DEFINING THE PROBLEM

def linreg(X, w, b):
  """The Linear Regression Model"""
  return torch.matmul(X,w) + b

In [32]:
# DEFINING THE LOSS FUNCTION

def squared_loss(y_hat, y):
  return (y_hat - y.reshape(y_hat.shape)) ** 2 /2

In [37]:
# DEFINING THE OPTIMIZATION ALGORITHM

def sgd(params, lr, batch_size):
  with torch.no_grad():
    for param in params:
      param -= lr * param.grad / batch_size
      param.grad.zero_()

In [38]:
# TRAINING MODEL

lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss
batch_size = 10

for epoch in range(num_epochs):
  for X, y in data_iter(batch_size,features,labels):
    l = loss(net(X,w,b),y)
    # Compute Gradient on l wrt to w,b
    l.sum().backward()
    sgd([w,b],lr,batch_size)
  with torch.no_grad():
    train_l = loss(net(features,w,b),labels)
    print(f"epoch {epoch+1}, loss {float(train_l.mean()):f}")
    

epoch 1, loss 0.027690
epoch 2, loss 0.000104
epoch 3, loss 0.000048


In [39]:
w

tensor([[ 1.9998],
        [-3.3999]], requires_grad=True)

In [40]:
true_w

tensor([ 2.0000, -3.4000])

In [41]:
print(f"error in estimating w: {true_w-w.reshape(true_w.shape)}")
print(f"error in estimating b: {true_b-b}")

error in estimating w: tensor([ 0.0002, -0.0001], grad_fn=<SubBackward0>)
error in estimating b: tensor([0.0002], grad_fn=<RsubBackward1>)


# LINEAR REGRESSION CONCISE IMPLEMENTATION

In [46]:
from torch.utils import data
from torch import nn 

In [42]:
# CREATE SYNTHETIC DATA

def synthetic_data(w, b, num_examples):
  """Generate y = Xw + b + noise"""
  X = torch.normal(0, 1, (num_examples, len(w)))
  y = torch.matmul(X, w) + b
  y+= torch.normal(0,0.01, y.shape)
  return X, y.reshape((-1, 1))

true_w = torch.tensor([2,-3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)
print(f"Features Shape :{features.shape}\n Labels Shape :{labels.shape}")

Features Shape :torch.Size([1000, 2])
 Labels Shape :torch.Size([1000, 1])


In [56]:
# READING DATASET

def load_array(data_arrays, batch_size, is_train=True):
  """Construct PYTORCH Data Iterator"""
  dataset = data.TensorDataset(*data_arrays)
  return data.DataLoader(dataset, batch_size, shuffle=is_train)

data_iter = load_array((features,labels), batch_size)

In [47]:
# DEFINING THE MODEL
net = nn.Sequential(nn.Linear(2,1))

In [48]:
# INITIALIZING THE MODEL PARAMATERS
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [49]:
# DEFINING THE LOSS FUNCTION
loss = nn.MSELoss()

In [51]:
# DEFINING THE OPTIMIZATION ALGORITHM
trainer = torch.optim.SGD(net.parameters(), lr=lr)

In [58]:
# TRAINING
num_epochs = 3
for epoch in range(num_epochs):
  for X, y in data_iter:
    l = loss(net(X),y)
    trainer.zero_grad()
    l.backward()
    trainer.step()
  l = loss(net(features),labels)
  print(f"epoch {epoch + 1} loss {l:f}")

epoch 1 loss 0.000101
epoch 2 loss 0.000102
epoch 3 loss 0.000101


In [59]:
w = net[0].weight.data
b = net[0].bias.data

In [60]:
print(f"error in estimating w: {true_w-w.reshape(true_w.shape)}")
print(f"error in estimating b: {true_b-b}")

error in estimating w: tensor([-0.0001,  0.0002])
error in estimating b: tensor([5.0068e-05])
